In [32]:
import os, sys
os.chdir("../")
path = os.getcwd()
sys.path.insert(0, path)
import GWGen
from GWGen.Utils import *
from GWGen.WFGenerator import *
import matplotlib.pyplot as plt



# set initial parameters
M = 100000
m = 1e1
mu = 3e-16
e0 = 0.001
p0 = 8#GetInitialP(M,e0)
Phi_phi0 = 0.
Phi_theta0 =0.
Phi_r0 = 0.


a=0.6 #SMBH Spin
Y0=1. #Initial Inclincation
qS=np.pi/4 #Sky Location Polar Angle in solar system barycenter coordinate system
phiS=0. #Sky Location Azimuthal Angle in solar system barycenter coordinate system
qK=1e-6 #Initial BH Spin Polar Angle in solar system barycenter coordinate system
phiK=0. #Initial BH Spin Azimuthal Angle in solar system barycenter coordinate system
dist=1. #Distance to source (Mpc)
mich=False #assume LISA long baseline response approximation

T=8 #LISA data run is 5 years. We set the max time to be longer because the proca cloud extends the inspiral time
dt=15 #time resolution in seconds

alphaval = alphavalue(M,mu)
print(r"alpha = {0}".format(alphaval))
print("initial p = {0}".format(p0))


use_gpu = False

# keyword arguments for inspiral generator (RunKerrGenericPn5Inspiral)
insp_kwargs = {
    "npoints": 110,  # we want a densely sampled trajectory
    "max_init_len": int(1e3),  # all of the trajectories will be well under len = 1000
    "dense_output":True
}

# keyword arguments for summation generator (AAKSummation)
sum_kwargs = {
    "use_gpu": use_gpu,  # GPU is availabel for this type of summation
    "pad_output": False,
}

WithoutProcaInspiralKwargs = insp_kwargs.copy()
WithoutProcaSumKwargs=sum_kwargs.copy()
withoutprocagen = EMRIWaveform(inspiral_kwargs=WithoutProcaInspiralKwargs, sum_kwargs=WithoutProcaSumKwargs, use_gpu=False)

alpha = 0.22449414184716673
initial p = 8


In [33]:
wv1 = withoutprocagen(M, m, a, p0, e0, Y0, qS, phiS, qK, phiK, dist,Phi_phi0=Phi_phi0, Phi_theta0=Phi_theta0, Phi_r0=Phi_r0, mich=mich, dt=dt, T=T)
wv2 = withoutprocagen(M, m, a, p0, e0+0.000005, Y0, qS, phiS, qK, phiK, dist,Phi_phi0=Phi_phi0, Phi_theta0=Phi_theta0, Phi_r0=Phi_r0, mich=mich, dt=dt, T=T)
td = np.arange(len(wv2))*dt

SystemError: <built-in function pyKerrGeoCoordinateFrequencies> returned a result with an error set

In [3]:
fd = np.fft.fftfreq(len(wv1), d = float(dt))
for i in np.arange(-5,5):
    phasearr = np.exp(-2*np.pi*1j*fd*i*dt)
    print(WaveformInnerProduct(td,wv1*phasearr,wv2))

0.024235897082725145
0.03726462168454676
0.06330419498663484
0.13738091469830258
0.5051620515872144
31.39840529841089
0.47391651316545996
0.1213755153928599
0.0525789724623428
0.02920922743921651


In [27]:
print(len(wv2))
print(len(wv1))
print(len(td))

1657887
1658050
1657887


In [28]:
WaveformInnerProduct(td,wv1,wv2, maximize=True)

RuntimeError: time domain has different length than the waveforms

In [29]:
Faithfulness(td,wv1,wv2,maximize=False)

0.9886189805315907

In [30]:
Faithfulness(td,wv1,wv2,maximize=True)

0.9886189805315907

In [ ]:
get_mismatch(wv1,wv2)a

In [ ]:
wv1f = np.fft.fft(wv1)
wv2f = np.fft.fft(wv2)
fd = np.fft.fftfreq(len(td),d=float(td[1]-td[0]))
fdlen = int(len(fd)/2-1)+1
fd = fd[1:fdlen]
wv1ft = wv1f[1:fdlen]
wv2ft = wv2f[1:fdlen]
wv2fstar = np.conjugate(wv2ft)
psd = LisaSensitivity(fd)
integrand = wv1ft*wv2fstar/psd

func1 = np.fft.ifft(wv1ft/psd)
func2 = np.fft.ifft(wv2fstar)


intfft = 4*np.real(np.fft.ifft(wv1ft/psd*wv2fstar))
conv = 4*np.real(np.convolve(np.hstack((func1[1:], func1)),func2, "valid"))
integral = 4*sp.integrate.simpson(integrand.real,x=fd)

print(intfft[0])
print(integral)
print(conv[0])

In [ ]:
td = np.arange(0,10,0.01)
w1 = 10
T1=5
w2 = 0.5
T2=0.5
fun = lambda x,w,T: np.exp(-w*x**2)*np.exp(-1j*T*x**2)
h1 = fun(td,w1,T1)
h2 = fun(td,w2,T2)
h1f = np.fft.fft(h1)
h2fs = np.conjugate(np.fft.fft(h2))
freqs = np.fft.fftfreq(len(td))

In [ ]:
convres = 4*np.real(np.convolve(np.hstack((h1[1:], h1)),h2, "valid"))
fftres = 4*np.real(
                np.fft.ifft(
                                np.fft.fft(h1)*np.fft.fft(h2)
                            ))
fig,ax = plt.subplots(2,2, figsize=(15,8))
ax[0,0].plot(convres, linestyle='dotted',label="conv")
ax[0,1].plot(fftres, linestyle='dashdot', label="fft")
ax[1,0].plot(convres-fftres, label="diff")
ax[0,0].legend()
ax[0,1].legend()
ax[1,0].legend()

In [ ]:


x=np.array([ 1.75+2.j , -0.25-0.5j, -0.75-0.5j,  0.25+0.j ])
h=np.array([ 2.5 +3.j  ,  0.75-0.75j,  0.  +0.j  , -1.25+0.75j])

X=np.fft.fft(x)
H=np.fft.fft(h)

Y=np.array(X)*np.array(H)
y=np.fft.ifft(Y)

isclose = lambda x: np.isclose(x,0)
print(np.all(list(map(isclose,np.abs(y - np.convolve(np.hstack((x[1:], x)),h, 'valid'))))))

In [ ]:
np.fft.fft(wv1)

In [ ]:
x = [0,0,0,0,1,1,1,0,0,0,0]
y = x

np.convolve(np.hstack((x[1:], x)),y, 'valid')